# KMeans Clustering:

**Cluster analysis** or **clustering** is the task of grouping a set of objects in such a way that objects in the same group (called a cluster) are more similar (in some sense) to each other than to those in other groups (clusters) _[Source: [Wikipedia page on clustering](https://en.wikipedia.org/wiki/Cluster_analysis)]_. 

**Kmeans** is the name of an algorithm for clustering data points into groups, also known as "k centroids." _[Source: [Wikipedia page on K-means](https://en.wikipedia.org/wiki/K-means_clustering)]_.

In [1]:
# Don't change this cell; just run it. 
from client.api.notebook import Notebook
ok = Notebook('Kmeans.ok')

Assignment: K-means
OK, version v1.14.15



For this exercise, we will be using the [World Happiness Report](https://worldhappiness.report/ed/2019/). I have downloaded the data and it is available in CSV format, as the next cell shows.

In [10]:
import pandas as pd
data1 = pd.read_csv('happiness-report.csv')
data1

,Country,Year,HappinessScore,LifeLadder,LogGDP,SocialSupport,HealthyLifeExpectancyAtBirth,FreedomToMakeLifeChoices,Generosity,PerceptionsOfCorruption,PositiveAffect,NegativeAffect,ConfidenceInNationalGovernment
0,Afghanistan,2018,3.203,2.694303,7.494588,0.507516,52.599998,0.373536,-0.084888,0.927606,0.424125,0.404904,0.364666
1,Albania,2018,4.719,5.004403,9.412399,0.683592,68.699997,0.824212,0.005385,0.899129,0.713300,0.318997,0.435338
2,Algeria,2018,5.211,5.043086,9.557952,0.798651,65.900002,0.583381,-0.172413,0.758704,0.591043,0.292946,NaN
3,Argentina,2018,6.086,5.792797,9.809972,0.899912,68.800003,0.845895,-0.206937,0.855255,0.820310,0.320502,0.261352
4,Armenia,2018,4.559,5.062449,9.119424,0.814449,66.900002,0.807644,-0.149109,0.676826,0.581488,0.454840,0.670828
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Venezuela,2018,4.707,5.005663,9.270281,0.886882,66.500000,0.610855,-0.176156,0.827560,0.759221,0.373658,0.260700
132,Vietnam,2018,5.175,5.295547,8.783416,0.831945,67.900002,0.909260,-0.039124,0.808423,0.692222,0.191061,NaN
133,Yemen,2018,3.380,3.057514,NaN,0.789422,56.700001,0.552726,NaN,0.792587,0.461114,0.314870,0.308151
134,Zambia,2018,4.107,4.041488,8.223958,0.717720,55.299999,0.790626,0.036644,0.810731,0.702698,0.350963,0.606715


<img align="right" style="padding-left:10px; height: 30%; width: 30%" src="figures/whr.png" >

**Origin:** The Happiness Score column is from Figure 2.6 in the [downloaded report](https://s3.amazonaws.com/happiness-report/2019/Chapter2OnlineData.xls); the other data columns are from Table 2.1 in the same report. If a country wasn't in either list, it wasn't included in the CSV file.

## Some observations about the data:

* **Quality** Some data points, e.g., Generosity for Yemen are unavailable. This is typically done using `dropna()` as shown below.
* **Normalization:** The columns have different ranges: some values are between 0. and 1., Generosity is between -0.33 and 0.49, etc. The data needs to be preprocessed so as to be uniform. The process of making all columns uniform in scale is referred to as **Feature Scaling**. Many data analysis libraries require it (see [_The Importance of Feature Scaling_](https://scikit-learn.org/stable/auto_examples/preprocessing/plot_scaling_importance.html)). Scikit-Learn offers an extensive library for [preprocessing data](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing). Pay particular attention to `*Scaler()` functions. Also Discretizers such as Binarizer, KBinsDiscretizer and QuantileTransformer.
* **Pre-normalization:** `LogGDP` is the logarithm of the GDP per capita. Here is a case where the data has gone through Feature Scaling _prior to publication!_ Taking a log of numbers whose range spans multiple orders of magnitude is a common technique for compressing the range. However, it still doesn't span the range [0. 1.] and a bit more Feature Scaling is required.

In [34]:
# Get a sense of data quality
data1_not_na = data1.dropna()
data1_good_or_na = pd.merge(data1, data1_not_na, indicator=True, how='left')
# _good_or_na has a new column '_merge' which has values 'left_only' or 'both'. Pick 'left_only'
data1_na = data1_good_or_na.query('_merge=="left_only"').drop('_merge', axis=1)
data1_na

,Country,Year,HappinessScore,LifeLadder,LogGDP,SocialSupport,HealthyLifeExpectancyAtBirth,FreedomToMakeLifeChoices,Generosity,PerceptionsOfCorruption,PositiveAffect,NegativeAffect,ConfidenceInNationalGovernment
2,Algeria,2018,5.211,5.043086,9.557952,0.798651,65.900002,0.583381,-0.172413,0.758704,0.591043,0.292946,NaN
18,Burundi,2018,3.775,3.775283,6.541033,0.484715,53.400002,0.646399,-0.019334,0.598608,0.666442,0.362767,NaN
19,Cambodia,2018,4.700,5.121838,8.253352,0.794605,61.599998,0.958305,0.033787,NaN,0.844593,0.414346,NaN
24,China,2018,5.191,5.131434,9.694376,0.787605,69.300003,0.895378,-0.174899,NaN,0.855784,0.189640,NaN
30,Cyprus,2018,6.046,6.276443,NaN,0.825573,73.699997,0.794215,NaN,0.848337,0.750122,0.298021,0.352440
35,Egypt,2018,4.166,4.005451,9.293960,0.758824,61.700001,0.681654,-0.222930,NaN,0.492261,0.285184,NaN
42,Gambia,2018,4.516,4.922099,7.376554,0.684800,55.000000,0.718729,NaN,0.691070,0.804012,0.379208,0.757543
58,Jordan,2018,4.906,4.638934,9.024435,0.799544,66.800003,0.762420,-0.183490,NaN,NaN,NaN,NaN
61,Kosovo,2018,6.100,6.391826,NaN,0.822407,65.149826,0.889737,NaN,0.922078,0.778271,0.170248,0.347547
63,Laos,2018,4.796,4.859402,8.813603,0.704738,58.700001,0.906661,0.140599,0.634240,0.852214,0.331883,NaN


### Fill

`data_na` has some rows _we don't want to reject._ For example, **China** and **Saudi Arabia**. Investigating China, we examine `data_na.loc[24]` and find the offending columns:
```
    PerceptionsOfCorruption                NaN
    ConfidenceInNationalGovernment         NaN
```
Examining `data1.mean()`, we find (among others), average values of _certain_ columns which we could "fill" with the averages below. We don't, however, want to fill in quantitative measures such as LogGDP.
```
    SocialSupport                        0.810544
    FreedomToMakeLifeChoices             0.784535
    Generosity                          -0.029086
    PerceptionsOfCorruption              0.731744
    PositiveAffect                       0.709628
    NegativeAffect                       0.293670
    ConfidenceInNationalGovernment       0.495120
```
**Note:** The decision to fill some of the columns _but not others_ may appear arbitrary to a reviewer. This is a good place to justify it.

In [43]:
# Fill some of the columns with the mean values and get a sense of data quality of the new data
means = data1.mean()
# Ref: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html
data2 = data1.fillna(value = { k:means[k] for k in means.keys() if k not in ['LogGDP']} )
data2

,Country,Year,HappinessScore,LifeLadder,LogGDP,SocialSupport,HealthyLifeExpectancyAtBirth,FreedomToMakeLifeChoices,Generosity,PerceptionsOfCorruption,PositiveAffect,NegativeAffect,ConfidenceInNationalGovernment
0,Afghanistan,2018,3.203,2.694303,7.494588,0.507516,52.599998,0.373536,-0.084888,0.927606,0.424125,0.404904,0.364666
1,Albania,2018,4.719,5.004403,9.412399,0.683592,68.699997,0.824212,0.005385,0.899129,0.713300,0.318997,0.435338
2,Algeria,2018,5.211,5.043086,9.557952,0.798651,65.900002,0.583381,-0.172413,0.758704,0.591043,0.292946,0.495120
3,Argentina,2018,6.086,5.792797,9.809972,0.899912,68.800003,0.845895,-0.206937,0.855255,0.820310,0.320502,0.261352
4,Armenia,2018,4.559,5.062449,9.119424,0.814449,66.900002,0.807644,-0.149109,0.676826,0.581488,0.454840,0.670828
...,...,...,...,...,...,...,...,...,...,...,...,...,...
131,Venezuela,2018,4.707,5.005663,9.270281,0.886882,66.500000,0.610855,-0.176156,0.827560,0.759221,0.373658,0.260700
132,Vietnam,2018,5.175,5.295547,8.783416,0.831945,67.900002,0.909260,-0.039124,0.808423,0.692222,0.191061,0.495120
133,Yemen,2018,3.380,3.057514,NaN,0.789422,56.700001,0.552726,-0.029086,0.792587,0.461114,0.314870,0.308151
134,Zambia,2018,4.107,4.041488,8.223958,0.717720,55.299999,0.790626,0.036644,0.810731,0.702698,0.350963,0.606715


In [88]:
# As above, we should get a sense of the quality of the data now.
data2 = data2.dropna()
pd_data = data2.drop(['Country', 'Year'], axis=1)
np_data = pd_countries.to_numpy()
# print (len(pd_data))
# print (numpy.isnan(pd_data).any()) # Any NaN values left?
np_cols = list(pd_data)
np_rows = list(data2['Country'])

def reconstruct_pd(np_data, np_rows, np_cols):
    return pd.concat([
        pd.DataFrame(np_rows, columns=['Country']), 
        pd.DataFrame(np_data, dtype='float32', columns=np_cols)], axis=1)

reconstruct_pd(np_data, np_rows, np_cols)

,Country,HappinessScore,LifeLadder,LogGDP,SocialSupport,HealthyLifeExpectancyAtBirth,FreedomToMakeLifeChoices,Generosity,PerceptionsOfCorruption,PositiveAffect,NegativeAffect,ConfidenceInNationalGovernment
0,Afghanistan,3.203,2.694303,7.494588,0.507516,52.599998,0.373536,-0.084888,0.927606,0.424125,0.404904,0.364666
1,Albania,4.719,5.004403,9.412399,0.683592,68.699997,0.824212,0.005385,0.899129,0.713300,0.318997,0.435338
2,Algeria,5.211,5.043086,9.557952,0.798651,65.900002,0.583381,-0.172413,0.758704,0.591043,0.292946,0.495120
3,Argentina,6.086,5.792797,9.809972,0.899912,68.800003,0.845895,-0.206937,0.855255,0.820310,0.320502,0.261352
4,Armenia,4.559,5.062449,9.119424,0.814449,66.900002,0.807644,-0.149109,0.676826,0.581488,0.454840,0.670828
...,...,...,...,...,...,...,...,...,...,...,...,...
122,Uzbekistan,6.174,6.205460,8.773365,0.920821,65.099998,0.969898,0.311695,0.520360,0.825422,0.208660,0.969356
123,Venezuela,4.707,5.005663,9.270281,0.886882,66.500000,0.610855,-0.176156,0.827560,0.759221,0.373658,0.260700
124,Vietnam,5.175,5.295547,8.783416,0.831945,67.900002,0.909260,-0.039124,0.808423,0.692222,0.191061,0.495120
125,Zambia,4.107,4.041488,8.223958,0.717720,55.299999,0.790626,0.036644,0.810731,0.702698,0.350963,0.606715


In [99]:
from sklearn.preprocessing import StandardScaler, RobustScaler
scaler = StandardScaler()
np_norm = scaler.fit_transform(np_data)
reconstruct_pd(np_norm, np_rows, np_cols)

,Country,HappinessScore,LifeLadder,LogGDP,SocialSupport,HealthyLifeExpectancyAtBirth,FreedomToMakeLifeChoices,Generosity,PerceptionsOfCorruption,PositiveAffect,NegativeAffect,ConfidenceInNationalGovernment
0,Afghanistan,-2.019505,-2.551520,-1.485569,-2.526380,-1.799854,-3.499283,-0.359735,1.087317,-2.628350,1.268071,-0.694985
1,Albania,-0.642061,-0.442753,0.137071,-1.044773,0.630625,0.337264,0.222222,0.925847,0.016003,0.268569,-0.323426
2,Algeria,-0.195028,-0.407441,0.260222,-0.076593,0.207934,-1.712902,-0.923980,0.129587,-1.101972,-0.034526,-0.009117
3,Argentina,0.600001,0.276930,0.473453,0.775471,0.645722,0.521843,-1.146541,0.677065,0.994557,0.286083,-1.238163
4,Armenia,-0.787438,-0.389766,-0.110812,0.056338,0.358895,0.196217,-0.773745,-0.334688,-1.189350,1.849055,0.914672
...,...,...,...,...,...,...,...,...,...,...,...,...
122,Uzbekistan,0.679958,0.653628,-0.403609,0.951415,0.087164,1.577465,2.196894,-1.221905,1.041302,-1.015151,2.484200
123,Venezuela,-0.652964,-0.441602,0.016826,0.665833,0.298510,-1.479014,-0.948107,0.520024,0.435936,0.904527,-1.241594
124,Vietnam,-0.227738,-0.176983,-0.395105,0.203562,0.509857,1.061261,-0.064710,0.411510,-0.176737,-1.219908,-0.009117
125,Zambia,-1.198127,-1.321746,-0.868457,-0.757594,-1.392258,0.051348,0.423740,0.424599,-0.080947,0.640482,0.577598


In [199]:
# This cell is referred to as "The Clustering Cell" in the discussion below. It's where the data are clustered.

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(np_norm)
kmeans.labels_

array([0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0], dtype=int32)

In [200]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import column_or_1d

class OrderedLabelEncoder(LabelEncoder):
# Reference: https://stackoverflow.com/a/51310759/653651
    def fit(self, y):
        y = column_or_1d(y, warn=True)
        self.classes_ = pd.Series(y).unique()
        return self

ole2 = OrderedLabelEncoder()
ole2.fit([0, 1])
#ole2.fit(['zero', 'one'])
#print (ole2.inverse_transform([0, 0, 0, 1, 1, 1]))
#ole3 = OrderedLabelEncoder()
#ole3.fit(['one', 'zero'])
#print (ole3.inverse_transform([0, 0, 0, 1, 1, 1]))
#ole4 = LabelEncoder()
#ole4.fit(['nil', 'one'])
#print (ole4.inverse_transform([0, 0, 0, 1, 1, 1]))
# print ( le.classes_)

def clustered_pd(np_data, np_rows, np_cols, labels, ole):
    return pd.concat([
        pd.DataFrame(np_rows, columns=['Country']), 
        pd.DataFrame(ole.inverse_transform(labels), columns=['Cluster']), 
        pd.DataFrame(np_data, dtype='float32', columns=np_cols)], axis=1)

# Uncomment the next line to help discover candidates for the labeling clusters
# print (clustered_pd(np_norm, np_rows, np_cols, kmeans.labels_, ole2).loc[:,['Country', 'Cluster']])
# I ran it once, then came up with the emojis below

In [201]:
# Replace labels with emojis
ole = OrderedLabelEncoder()
ole.fit(['\U0001F626', '\U0001F601'])

clustered_pd(np_norm, np_rows, np_cols, kmeans.labels_, ole)

,Country,Cluster,HappinessScore,LifeLadder,LogGDP,SocialSupport,HealthyLifeExpectancyAtBirth,FreedomToMakeLifeChoices,Generosity,PerceptionsOfCorruption,PositiveAffect,NegativeAffect,ConfidenceInNationalGovernment
0,Afghanistan,😦,-2.019505,-2.551520,-1.485569,-2.526380,-1.799854,-3.499283,-0.359735,1.087317,-2.628350,1.268071,-0.694985
1,Albania,😦,-0.642061,-0.442753,0.137071,-1.044773,0.630625,0.337264,0.222222,0.925847,0.016003,0.268569,-0.323426
2,Algeria,😦,-0.195028,-0.407441,0.260222,-0.076593,0.207934,-1.712902,-0.923980,0.129587,-1.101972,-0.034526,-0.009117
3,Argentina,😁,0.600001,0.276930,0.473453,0.775471,0.645722,0.521843,-1.146541,0.677065,0.994557,0.286083,-1.238163
4,Armenia,😦,-0.787438,-0.389766,-0.110812,0.056338,0.358895,0.196217,-0.773745,-0.334688,-1.189350,1.849055,0.914672
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Uzbekistan,😁,0.679958,0.653628,-0.403609,0.951415,0.087164,1.577465,2.196894,-1.221905,1.041302,-1.015151,2.484200
123,Venezuela,😦,-0.652964,-0.441602,0.016826,0.665833,0.298510,-1.479014,-0.948107,0.520024,0.435936,0.904527,-1.241594
124,Vietnam,😁,-0.227738,-0.176983,-0.395105,0.203562,0.509857,1.061261,-0.064710,0.411510,-0.176737,-1.219908,-0.009117
125,Zambia,😦,-1.198127,-1.321746,-0.868457,-0.757594,-1.392258,0.051348,0.423740,0.424599,-0.080947,0.640482,0.577598


<img align="right" style="padding-left:10px; height: 50%; width: 50%" src="figures/image001.png" >

## Judgment Factors

### Treatment of Outliers

Outliers can influence the clustering result. Removing outliers is an essential part of prepping the data for cluster analysis.

Outliers represent experimental error or other spurious results (referred to as _artifacts_ in microbiology literature). In other applications, they are _most valuable_ such as when looking for criminal activity patterns in e-commerce data See [Novelty and Outlier Detection](https://scikit-learn.org/stable/modules/outlier_detection.html) library in Scikit-Learn.

_In other words, removing outliers isn't always desirable!_


<img align="right" style="padding-left:10px; height: 50%; width: 50%" src="figures/image003.png" >

### Randomness of seed selection

As the accompanying diagrams show, the k-means algorithm is subject to the randomness of seed selection, which can play a significant role in the results of clustering. 

<img align="right" style="padding-left:10px; height: 50%; width: 50%" src="figures/image005.png" >

To counter the effects of this randomness, modifications to the algorithm have been proposed, the most well-known of them being [kmeans++](https://en.wikipedia.org/wiki/K-means%2B%2B). K-means++ implementations are available in most libraries (including Scikit-Learn).

_Most of the time you want to use K-means++._

### Data suitable for K-means 

The K-means algorithm works well for data that are "ellipsoidal" in shape.

<img align="right" style="padding-left:10px; height: 50%; width: 50%" src="figures/image007.png" >

The data shown here don't lend themselves to be clustered into ellipsoids. Figure B represents the best that can be achieved with this algorithm.

### Parity of Cluster Sizes

<img align="right" style="padding-left:10px; height: 70%; width: 70%" src="figures/500px-mouse.png" >

There is no assumption in standard K-means algorithms that assumes an equal number of points in each cluster. However, certain standard algorithms do have a tendency towards _equalizing the spatial variance of clusters,_ which can result in a (rough) tendency towards equality of cluster sizes in cases where there is overlap between the clusters &mdash; [source](https://stats.stackexchange.com/a/326715).

Notice how the clusters chosen by K-means bleed into each other more than in the original data. Other methods, such as the [Expectation–maximization algorithm](https://en.wikipedia.org/wiki/Expectation%E2%80%93maximization_algorithm) labeled EM Clustering in the figure, fare better in this case.

### Assigning Cluster Labels

We were able to come up with emojis for the values 0 and 1 with some confidence. But this approach isn't fool-proof. What could possibly go wrong? The allocation of labels 0 and 1 is arbitrary. The labels could be switched the next time the clustering cell is run! To counter this possibility, the labeling was done as a separate step _after_ the clustering had been run.

### For further reading

[k-Means Advantages and Disadvantages](https://developers.google.com/machine-learning/clustering/algorithm/advantages-disadvantages) &mdash; from Machine Learning Crash Course by Google.

In [202]:
from sklearn.cluster import KMeans
n_clusters = 5
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(np_norm)
kmeans.labels_

array([2, 1, 3, 4, 1, 0, 0, 1, 1, 3, 4, 2, 4, 3, 1, 4, 3, 2, 2, 1, 2, 0,
       2, 4, 4, 4, 2, 2, 4, 3, 4, 0, 4, 4, 3, 4, 4, 1, 0, 4, 3, 2, 3, 0,
       3, 4, 2, 2, 4, 1, 1, 2, 0, 4, 3, 2, 4, 3, 4, 1, 1, 1, 3, 3, 2, 3,
       0, 3, 2, 2, 1, 2, 3, 4, 4, 3, 3, 3, 2, 1, 1, 1, 2, 0, 0, 4, 2, 1,
       0, 2, 4, 4, 1, 4, 4, 3, 1, 4, 1, 3, 2, 4, 4, 1, 3, 4, 1, 1, 0, 0,
       1, 4, 2, 3, 3, 1, 2, 3, 4, 0, 4, 4, 0, 3, 4, 1, 1], dtype=int32)

## TO-DO

Replace arbitrary numeric labels with descriptive numerical labels for categories.

In [247]:
# Replace numeric labels with descriptive labels
ole = OrderedLabelEncoder()
ole.fit(list(range(n_clusters)))
cpd = clustered_pd(np_norm, np_rows, np_cols, kmeans.labels_, ole)

means = cpd.groupby('Cluster')['HappinessScore'].mean()
new_indexes = sorted(range(n_clusters), key=lambda k: means[k])

dd = {new_indexes[i]:i for i in range(5)}
ole = OrderedLabelEncoder()
ole.fit([dd[i] for i in range(5)])

cpd = clustered_pd(np_norm, np_rows, np_cols, kmeans.labels_, ole)
sorted_indexes = cpd.groupby('Cluster')['HappinessScore'].mean().sort_values(ascending=True)

# Confirm that the resulting cluster annotations are in order of average happiness score.

# print (sorted_indexes) 
# Cluster
# 0   -0.934783
# 1   -0.809009
# 2   -0.188633
# 3    0.698260
# 4    1.649559
# Name: HappinessScore, dtype: float32
cpd

,Country,Cluster,HappinessScore,LifeLadder,LogGDP,SocialSupport,HealthyLifeExpectancyAtBirth,FreedomToMakeLifeChoices,Generosity,PerceptionsOfCorruption,PositiveAffect,NegativeAffect,ConfidenceInNationalGovernment
0,Afghanistan,0,-2.019505,-2.551520,-1.485569,-2.526380,-1.799854,-3.499283,-0.359735,1.087317,-2.628350,1.268071,-0.694985
1,Albania,1,-0.642061,-0.442753,0.137071,-1.044773,0.630625,0.337264,0.222222,0.925847,0.016003,0.268569,-0.323426
2,Algeria,2,-0.195028,-0.407441,0.260222,-0.076593,0.207934,-1.712902,-0.923980,0.129587,-1.101972,-0.034526,-0.009117
3,Argentina,3,0.600001,0.276930,0.473453,0.775471,0.645722,0.521843,-1.146541,0.677065,0.994557,0.286083,-1.238163
4,Armenia,1,-0.787438,-0.389766,-0.110812,0.056338,0.358895,0.196217,-0.773745,-0.334688,-1.189350,1.849055,0.914672
...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,Uzbekistan,4,0.679958,0.653628,-0.403609,0.951415,0.087164,1.577465,2.196894,-1.221905,1.041302,-1.015151,2.484200
123,Venezuela,2,-0.652964,-0.441602,0.016826,0.665833,0.298510,-1.479014,-0.948107,0.520024,0.435936,0.904527,-1.241594
124,Vietnam,3,-0.227738,-0.176983,-0.395105,0.203562,0.509857,1.061261,-0.064710,0.411510,-0.176737,-1.219908,-0.009117
125,Zambia,1,-1.198127,-1.321746,-0.868457,-0.757594,-1.392258,0.051348,0.423740,0.424599,-0.080947,0.640482,0.577598


Draw scatter plots of 

1. Happiness-Score vs. LogGDP, 
2. Happiness-Score vs. SocialSupport, 
3. Happiness-Score vs. HealthyLifeExpectancyAtBirth and
4. Happiness-Score vs. PerceptionsOfCorruption. 

In each case, label the points with Country and color them according to cluster [0 = red, 1 = orange, 2 = yellow, 3 = lime green, 4 = dark green]

By visual inspection, how correlated is the happiness score with each parameter?

**Your answer:** ______
<hr style="border-top: 1px dotted #ddd" />
<hr style="border-top: 1px dotted #ddd" />
<hr style="border-top: 1px dotted #ddd" />
<hr style="border-top: 1px dotted #ddd" />

